In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense, Activation
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from pprint import pprint
import numpy as n

### Trainingsdaten
Diesmal holen wir uns die Trainingsdaten aus einer csv-Datei.

In [2]:
# Laden der iris.csv-Datei
iris_data = np.loadtxt("iris.csv",delimiter=",",dtype="str",skiprows=1)
# Wir erstellen die Kategorien:
iris_label_array = ["Iris-setosa", "Iris-versicolor", "Iris-virginica"]

Wir ersetzen jetzt in de Eingabedaten das Label durch einen Index, d.h. in eine Zahl

In [3]:
label_index = 0
for label in iris_label_array :
    print(label)
# Wenn eines der Label innerhalb von iris_data gefunden wird,
# wird dieses durch den label_index ersetzt,
# z. B. werden alle Einträge von Iris-versicolor durch den Wert 1
# ersetzt.
    iris_data[np.where(iris_data[:,4]==label), 4] = label_index
    label_index = label_index + 1

Iris-setosa
Iris-versicolor
Iris-virginica


Trainingsdaten vorbereiten

In [4]:
iris_data = iris_data.astype("float32")

input_data = iris_data[:,0:4] # Spalten 0 bis 4 werden extrahiert

output_data = iris_data[:,4].reshape(-1, 1)
print(output_data[45:55])

# Die 4. Spalte wird extrahiert output_data = output_data.reshape(-1, 1)
# Array von 1D-Array 
output_data = tf.keras.utils.to_categorical(output_data,3)  # ONe-Hot-Encoding
print(output_data[45:55])

[[0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]]


Train-Test Splitting 80:20

In [5]:
X_train, X_test, y_train, y_test = train_test_split(input_data,output_data, test_size=0.20)



### Modell erstellen mit Keras

Drei Schichten anlegen

In [6]:
iris_model = Sequential()
iris_model.add(Dense(5, input_shape=(4,), activation="tanh"))
iris_model.add(Dense(24, activation="relu"))
iris_model.add(Dense(3, activation="softmax"))

Modell kompilieren und Training starten (fit)

In [7]:
sgd = SGD(learning_rate=0.001)

iris_model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])
iris_model.fit(x=X_train, y=y_train, batch_size=10,
epochs=500, verbose=1)

Epoch 1/500
12/12 [==============================] - 0s 1ms/step - loss: 1.1535 - accuracy: 0.3361
Epoch 2/500
12/12 [==============================] - 0s 1ms/step - loss: 1.1502 - accuracy: 0.3361
Epoch 3/500
12/12 [==============================] - 0s 1ms/step - loss: 1.1471 - accuracy: 0.3361
Epoch 4/500
12/12 [==============================] - 0s 934us/step - loss: 1.1441 - accuracy: 0.3361
Epoch 5/500
12/12 [==============================] - 0s 1ms/step - loss: 1.1414 - accuracy: 0.3361
Epoch 6/500
12/12 [==============================] - 0s 934us/step - loss: 1.1386 - accuracy: 0.3361
Epoch 7/500
12/12 [==============================] - 0s 794us/step - loss: 1.1363 - accuracy: 0.3361
Epoch 8/500
12/12 [==============================] - 0s 740us/step - loss: 1.1339 - accuracy: 0.3361
Epoch 9/500
12/12 [==============================] - 0s 1ms/step - loss: 1.1319 - accuracy: 0.3361
Epoch 10/500
12/12 [==============================] - 0s 902us/step - loss: 1.1299 - accuracy: 0.3361

### Evaluation auf Testdaten

In [8]:
# Evaluation auf Testdaten
evaluation_results = iris_model.evaluate(X_test, y_test)
print("Loss: {}".format(evaluation_results[0]))
print("Accuracy: {}".format(evaluation_results[1]))
#print("Mean Absolute Error: {}".format(evaluation_results[2]))

1/1 [==============================] - 0s 168ms/step - loss: 0.5778 - accuracy: 0.5667
Loss: 0.5778099298477173
Accuracy: 0.5666666626930237


Einzeltests

In [9]:
test_data = np.array([
    [5.1,3.5,1.4,0.2],  # Zeile 1 Setosa
    [5.9,3.,5.1,1.8],   # Virginica 
    [4.9,3.,1.4,0.2],   # Setosa
    [5.8,2.7,4.1,1.]])  # Versicolor
predictions = iris_model.predict(test_data)
predictions

array([[0.91265994, 0.03702855, 0.05031146],
       [0.06695302, 0.44169423, 0.49135274],
       [0.9006653 , 0.04270158, 0.0566331 ],
       [0.0833009 , 0.43755525, 0.4791439 ]], dtype=float32)

In [10]:
index_max_predictions = np.argmax(predictions, axis=1)
for i in index_max_predictions:
    print("Iris mit den Eigenschaften {} gehört zur Klasse: {}".format(
test_data[i], iris_label_array[i]))

Iris mit den Eigenschaften [5.1 3.5 1.4 0.2] gehört zur Klasse: Iris-setosa
Iris mit den Eigenschaften [4.9 3.  1.4 0.2] gehört zur Klasse: Iris-virginica
Iris mit den Eigenschaften [5.1 3.5 1.4 0.2] gehört zur Klasse: Iris-setosa
Iris mit den Eigenschaften [4.9 3.  1.4 0.2] gehört zur Klasse: Iris-virginica
